# Verify Installation

In [1]:
import tokentoss
print(f"tokentoss version: {tokentoss.__version__}")
print(f"Config path: {tokentoss.get_config_path()}")

tokentoss version: 0.1.0
Config path: /Users/nicholasgrundl/Library/Application Support/tokentoss/client_secrets.json


# Check for Existing Config

In [2]:
from pathlib import Path

config_path = tokentoss.get_config_path()
if config_path.exists():
    print(f"Config exists at: {config_path}")
    perms = oct(config_path.stat().st_mode & 0o777)
    print(f"Permissions: {perms}")
else:
    print("No config found — will use ConfigureWidget next")

No config found — will use ConfigureWidget next


# ConfigureWidget

In [3]:
from tokentoss import ConfigureWidget

cw = ConfigureWidget()
display(cw)

# Verify Config Was Saved

In [4]:
import json, os

config_path = tokentoss.get_config_path()
assert config_path.exists(), "Config file not created!"

perms = oct(config_path.stat().st_mode & 0o777)
print(f"File: {config_path}")
print(f"Permissions: {perms}")
assert perms == "0o600", f"Expected 0o600, got {perms}"

with open(config_path) as f:
    data = json.load(f)

print(f"Has 'installed' key: {'installed' in data}")
print(f"Client ID ends with: ...{data['installed']['client_id'][-20:]}")
print("Config saved correctly!")

File: /Users/nicholasgrundl/Library/Application Support/tokentoss/client_secrets.json
Permissions: 0o600
Has 'installed' key: True
Client ID ends with: ...oogleusercontent.com
Config saved correctly!


# GoogleAuthWidget — Fresh Auth

In [5]:
from tokentoss import GoogleAuthWidget

widget = GoogleAuthWidget()
display(widget)